In [1]:
# ! pip install ipympl
#! pip install scikit-learn
#! pip install seaborn

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns


# 1. 데이터 로딩 및 전처리

In [3]:
# 예시 경로, 실제 경로로 수정 필요

# 게임에 참가한 사람들의 고유 정보
participants = pd.read_csv("data/user/participants.csv")
# 시간 흐름에 따라 변화하는 실시간 스탯 (골드, 딜량 등) 
participants_frames = pd.read_csv("data/user/participant_frames.csv")
# 게임 단위의 결과 (승패 등)
matches = pd.read_csv("data/user/matches.csv")
# 킬 이벤트 정보 (누가 누구를 죽였는지)
champ_kill = pd.read_csv('data/user/events__CHAMPION_KILL.csv')

In [4]:
# 가장 마지막 프레임만 사용 (게임 종료 시점 상태)
# 게임 종료 시점 기준으로 정보 하나로 묶기
latest_frames = participants_frames.sort_values("frame_timestamp").drop_duplicates(
    subset=["game_id", "participant_id"], keep='last'
)

merged_df = pd.merge(participants, latest_frames, on=["game_id", "participant_id"], how="left")

## 2. KDA 및 주요 파생 지표 생성

### KDA 계산

컬럼 | 의미
---- | ----
kills | 해당 경기에서 몇 번 킬 했는가
deaths | 몇 번 죽었는가
assists | 어시스트 수
KDA | (킬 + 어시스트) / 데스 (생존 기반 전투 영향력)

In [5]:
# kills 수 계산 (킬한 사람 기준)
kills = champ_kill.groupby(["game_id", "killer_id"]).size().reset_index(name="kills")
kills.rename(columns={"killer_id": "participant_id"}, inplace=True)

In [6]:
# deaths 수 계산 (죽은 사람 기준)
deaths = champ_kill.groupby(["game_id", "victim_id"]).size().reset_index(name="deaths")
deaths.rename(columns={"victim_id": "participant_id"}, inplace=True)

In [7]:
# assists 수 계산
assist_df = pd.read_csv("data/top1000/event_assists.csv")

# 게임 ID + 어시스트 참가자 ID로 그룹화
assists = assist_df.groupby(["game_id", "participant_id"]).size().reset_index(name="assists")


In [8]:
# KDA 구하기

# 기본 participant list (merged_df)에서 가져오자
kda_base = merged_df[["game_id", "participant_id", "puuid"]].drop_duplicates()

# 병합
kda_df = (
    kda_base
    .merge(kills, on=["game_id", "participant_id"], how="left")
    .merge(deaths, on=["game_id", "participant_id"], how="left")
)

# 어시스트 컬럼 있으면 병합
if 'assists' in locals():
    kda_df = kda_df.merge(assists, on=["game_id", "participant_id"], how="left")

# 결측값 0으로 채우기
kda_df[["kills", "deaths", "assists"]] = kda_df[["kills", "deaths", "assists"]].fillna(0).astype(int)

# KDA 계산
# 분모가 0이 되면 오류가 나기 때문에, 완벽 생존은 1로 나눈 값을 최대치로 보정
kda_df["KDA"] = (kda_df["kills"] + kda_df["assists"]) / kda_df["deaths"].replace(0, 1)


In [9]:
# merged_df에 병합

merged_df = pd.merge(merged_df, kda_df[["game_id", "participant_id", "kills", "deaths", "assists", "KDA"]],
                     on=["game_id", "participant_id"], how="left")
merged_df.head()

,game_id,participant_id,puuid,frame_timestamp,current_gold,gold_per_second,jungle_minions_killed,level,minions_killed,time_enemy_spent_controlled,...,totalDamageDone,totalDamageDoneToChampions,totalDamageTaken,trueDamageDone,trueDamageDoneToChampions,trueDamageTaken,kills,deaths,assists,KDA
0,7494844141,1,FjQNTm2bigLzOTT-Z3OHxkOUv03NA94H5MtpbZvZ5rReDq...,2109058,41,0,192,16,27,808201,...,282328,30644,53658,80762,2110,965,7,11,0,0.636364
1,7494844141,2,GIMAB6FWhOn1fGc3QwGL1N9-4Zp9bLF7OQFZUMveLZRAN7...,2109058,54,0,4,16,184,66402,...,118735,23226,30316,13692,2111,746,3,8,0,0.375000
2,7494844141,3,GUtnltQQ4cxM_6Pf1Kl5OqzPzs-NFR8-0xcb3g_oi2uVP8...,2109058,697,0,0,16,191,292153,...,109884,11419,38977,878,0,911,0,10,0,0.000000
3,7494844141,4,IRp95UwU1QKgaiSJCTjO3aYlIiVMopikQs7_bBglbHxZNu...,2109058,1466,0,32,18,341,28975,...,402103,68445,25425,9706,504,1067,11,7,0,1.571429
4,7494844141,5,-d5MIcv7Gtetq5g1xLaolc5qUozE9oX3ogQQJCLK2RCUv_...,2109058,197,50,0,14,18,137149,...,18499,7148,24831,4038,0,562,0,10,0,0.000000


## 유사도 기반 파생 지표 생성 (예시: 시야 점수, 미니언 수, 골드 등)

### 성장 및 경제 지표        

| 지표 | 의미 | 해석 |  
| ---- | ---- | ---- |   
| CS/Min | 분당 미니언 수급량 | 파밍 능력 |     
| Gold/Min | 분당 골드 | 성장 속도 |        
| XP/Min | 분당 경험치 | 레벨 성장 속도 |       




In [10]:
# 성장 및 경제 지표

# 분당 라인 몬스터 
merged_df['lane_cs_per_min'] = merged_df['minions_killed'] / (merged_df["frame_timestamp"] / 60000)

# 분당 정글 몬스터
merged_df['jungle_cs_per_min'] = merged_df['jungle_minions_killed'] / (merged_df["frame_timestamp"] / 60000)

# # CS (미니언 + 정글)
# merged_df["cs"] = merged_df["minions_killed"] + merged_df["jungle_minions_killed"]

# # 분당 CS
# merged_df["cs_per_min"] = merged_df["cs"] / (merged_df["frame_timestamp"] / 60000)

# jungle 성향 분석
merged_df['jungle_ratio'] = merged_df['jungle_minions_killed'] / (merged_df['jungle_minions_killed'] + merged_df['minions_killed'])

# 분당 골드
merged_df["gold_per_min"] = merged_df["total_gold"] / (merged_df["frame_timestamp"] / 60000)

# 분당 경험치
merged_df["xp_per_min"] = merged_df["xp"] / (merged_df["frame_timestamp"] / 60000)

### 전투 능력 지표

지표 | 의미 | 해석
---- | ---- | ----
Damage/Min | 분당 딜량 | 전투 기여도
Damage Taken/Min | 분당 받은 피해 | 탱킹 능력 또는 위험 노출
KA/Min | 분당 킬 관여 수 | 싸움 참여율 (적극적인지 여부)
Damage Efficiency | 피해 교환 비율 | 딜 효율 (딜을 얼마나 잘 넣고 덜 맞았는지)

In [11]:
# 전투력 관련 지표
# 분당 피해량
merged_df["damage_per_min"] = merged_df["totalDamageDoneToChampions"] / (merged_df["frame_timestamp"] / 60000)

# 분당 받은 피해량
merged_df["damage_taken_per_min"] = merged_df["totalDamageTaken"] / (merged_df["frame_timestamp"] / 60000)

# 분당 킬 관여율 (K+A)/시간
merged_df["ka_per_min"] = (merged_df["kills"] + merged_df["assists"]) / (merged_df["frame_timestamp"] / 60000)

# 피해 교환 효율 (딜량 / 받은 피해량)
merged_df["damage_efficiency"] = merged_df["totalDamageDoneToChampions"] / merged_df["totalDamageTaken"].replace(0, 1)


### 생존력 지표

지표 | 의미 | 해석
---- | ---- | ----
survivability | 생존률 | 
healthregen_per_min | 분당 체력 재생 | 


In [12]:
# 생존력 지표
# 생존률 (1 - 데스 / 평균 생존시간) → 데스 적을수록 높음
merged_df["survivability"] = 1 / merged_df["deaths"].replace(0, 1)

# 1. healthregen의 총합을 참가자별로 groupby
agg_df = participants_frames.groupby(['game_id', 'participant_id']).agg({
    'healthregen': 'sum',
    'frame_timestamp': 'max'  # 해당 플레이어의 게임 시간 (ms)
}).reset_index()

# 2. 게임 시간(ms)를 분 단위로 변환
agg_df['game_time_min'] = agg_df['frame_timestamp'] / 60000

# 3. 분당 체력 재생량 계산
agg_df['healthregen_per_min'] = agg_df['healthregen'] / agg_df['game_time_min']

# 4. merged_df에 병합 (game_id + participant_id 기준)
merged_df = merged_df.merge(
    agg_df[['game_id', 'participant_id', 'healthregen_per_min']],
    on=['game_id', 'participant_id'],
    how='left'
)

# # 분당 체력 재생
# merged_df["healthregen_per_min"] = merged_df["healthregen"] / (merged_df["frame_timestamp"] / 60000)


### 시야 장악 지표

지표 | 의미 | 해석
---- | ---- | ----
Wards Placed/Min | 분당 설치 | 시야 제공 적극성
Wards Killed/Min | 분당 제거 | 상대 시야 차단 능력
Vision Score Est. | 설치+제거 총합 | 시야 장악 종합 평가 (approx.)

In [13]:
# 포지션 제어 / 시야 장악 지표
wards_placed = pd.read_csv("data/top1000/events__WARD_PLACED.csv")
wards_killed = pd.read_csv("data/top1000/events__WARD_KILL.csv")

## 와드 설치 수
ward_placed_count = (
    wards_placed
    .groupby(["game_id", "creator_id"])
    .size()
    .reset_index(name="wards_placed")
    .rename(columns={"creator_id": "participant_id"})
)

## 와드 제거 수
ward_kill_count = (
    wards_killed
    .groupby(["game_id", "killer_id"])
    .size()
    .reset_index(name="wards_killed")
    .rename(columns={"killer_id": "participant_id"})
)

## merged_df에 와드 수 붙이기
merged_df = (
    merged_df
    .merge(ward_placed_count, on=["game_id", "participant_id"], how="left")
    .merge(ward_kill_count, on=["game_id", "participant_id"], how="left")
)

# 결측치는 0으로 처리
merged_df[["wards_placed", "wards_killed"]] = merged_df[["wards_placed", "wards_killed"]].fillna(0).astype(int)

## 시야 제어 파생 지표 추가
# 분당 와드 설치
merged_df["wards_placed_per_min"] = merged_df["wards_placed"] / (merged_df["frame_timestamp"] / 60000)

# 분당 와드 제거
merged_df["wards_killed_per_min"] = merged_df["wards_killed"] / (merged_df["frame_timestamp"] / 60000)

# 종합 시야 지표 (설치 + 제거)
merged_df["vision_score_est"] = merged_df["wards_placed"] + merged_df["wards_killed"]


### 종합 능력치 지표
기본 아이템 + 능력치 요약

In [14]:
# 아이템/스탯 관련 종합 능력치
# 전체 공격 스탯
merged_df["offensive_score"] = merged_df["attackdamage"] + merged_df["attackspeed"]

# 전체 방어 스탯
merged_df["defensive_score"] = merged_df["armor"] + merged_df["magicresist"]

# 전체 마법 능력치
merged_df["magic_score"] = merged_df["abilitypower"] + merged_df["magicpen"] + merged_df["magicpenpercent"]


생존 또는 스킬 활용 능력

In [15]:
# 기타 유틸성 스탯
# 만능 흡혈 + 물리 흡혈 + 주문 흡혈
merged_df["total_vamp"] = merged_df[["omnivamp", "physicalvamp", "spellvamp"]].sum(axis=1)

# 스킬 쿨다운 영향력
merged_df["haste_score"] = merged_df["cooldownreduction"] + merged_df["abilityhaste"]


In [16]:
merged_df

,game_id,participant_id,puuid,frame_timestamp,current_gold,gold_per_second,jungle_minions_killed,level,minions_killed,time_enemy_spent_controlled,...,wards_placed,wards_killed,wards_placed_per_min,wards_killed_per_min,vision_score_est,offensive_score,defensive_score,magic_score,total_vamp,haste_score
0,7494844141,1,FjQNTm2bigLzOTT-Z3OHxkOUv03NA94H5MtpbZvZ5rReDq...,2109058,41,0,192,16,27,808201,...,0,0,0.0,0.0,0,519,172,0,0,0
1,7494844141,2,GIMAB6FWhOn1fGc3QwGL1N9-4Zp9bLF7OQFZUMveLZRAN7...,2109058,54,0,4,16,184,66402,...,0,0,0.0,0.0,0,220,138,403,0,0
2,7494844141,3,GUtnltQQ4cxM_6Pf1Kl5OqzPzs-NFR8-0xcb3g_oi2uVP8...,2109058,697,0,0,16,191,292153,...,0,0,0.0,0.0,0,242,208,194,0,0
3,7494844141,4,IRp95UwU1QKgaiSJCTjO3aYlIiVMopikQs7_bBglbHxZNu...,2109058,1466,0,32,18,341,28975,...,0,0,0.0,0.0,0,622,157,0,0,0
4,7494844141,5,-d5MIcv7Gtetq5g1xLaolc5qUozE9oX3ogQQJCLK2RCUv_...,2109058,197,50,0,14,18,137149,...,0,0,0.0,0.0,0,213,127,147,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7276335666,6,rf8UusUjnzDhoSdnw1FjTTkHEwM4-h5Rh3xA9fe6qrb91M...,1429418,1420,0,8,15,167,328787,...,0,0,0.0,0.0,0,371,178,212,0,0
96,7276335666,7,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1429418,489,0,136,13,15,441586,...,0,0,0.0,0.0,0,423,165,24,0,0
97,7276335666,8,2-AUndHRg0e4w0tIgiUSCVkGt6TwJpKXB92wFTXGR089Gy...,1429418,890,0,16,16,191,134230,...,0,0,0.0,0.0,0,488,148,24,0,0
98,7276335666,9,unLyIkTbvbGsOd4U5_sHbMP_H6WLbQi3UmQgFaXdAMd9Eu...,1429418,960,0,0,13,178,47827,...,0,0,0.0,0.0,0,446,120,60,0,0


In [17]:
merged_df = merged_df.drop(columns = ['game_id', 'participant_id'])

In [25]:
target_puuid = (
    "-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFXsxfHwXECJ_bWG0f57TpkVxUgYiRdb85Q"
)
X_all = merged_df[merged_df["puuid"] == target_puuid].fillna(0)
X_all.to_csv("data/result/user_all.csv")
X_all

,puuid,frame_timestamp,current_gold,gold_per_second,jungle_minions_killed,level,minions_killed,time_enemy_spent_controlled,total_gold,xp,...,wards_placed,wards_killed,wards_placed_per_min,wards_killed_per_min,vision_score_est,offensive_score,defensive_score,magic_score,total_vamp,haste_score
5,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,2109058,1854,50,0,15,27,192025,9928,14165,...,0,0,0.0,0.0,0,290,297,73,0,0
16,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1557593,499,0,105,12,17,196813,10338,9662,...,0,0,0.0,0.0,0,233,190,304,0,0
25,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,2038851,397,0,20,18,254,233832,16991,18776,...,0,0,0.0,0.0,0,669,199,0,0,0
31,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1563034,1573,0,153,16,21,200243,16437,15699,...,0,0,0.0,0.0,0,317,154,746,0,0
46,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,2468923,4876,0,269,18,86,538127,25905,29530,...,0,0,0.0,0.0,0,677,174,0,0,0
59,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1831942,1337,50,0,13,31,251263,9157,11206,...,0,0,0.0,0.0,0,218,233,0,0,0
61,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,965237,1129,0,101,11,13,168825,8049,7580,...,0,0,0.0,0.0,0,342,121,0,0,0
76,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,2076960,1024,0,133,16,29,660477,13103,15297,...,0,0,0.0,0.0,0,491,257,0,0,0
81,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,958008,57,0,80,9,6,261910,5205,5534,...,0,0,0.0,0.0,0,329,113,0,0,0
96,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1429418,489,0,136,13,15,441586,8405,10870,...,0,0,0.0,0.0,0,423,165,24,0,0


In [19]:
grouped_df = merged_df.groupby('puuid').mean().reset_index()
grouped_df

,puuid,frame_timestamp,current_gold,gold_per_second,jungle_minions_killed,level,minions_killed,time_enemy_spent_controlled,total_gold,xp,...,wards_placed,wards_killed,wards_placed_per_min,wards_killed_per_min,vision_score_est,offensive_score,defensive_score,magic_score,total_vamp,haste_score
0,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1699902.4,1323.5,10.0,99.7,14.1,49.9,314510.1,12351.8,13831.9,...,0.0,0.0,0.0,0.0,0.0,398.9,190.3,114.7,0.0,0.0
1,-J6KMZ7IDQZNRHXtuMzSQQRkDwinSGI62fo5qD6C700Hzr...,958008.0,1473.0,50.0,0.0,7.0,20.0,44899.0,3823.0,3980.0,...,0.0,0.0,0.0,0.0,0.0,182.0,99.0,29.0,0.0,0.0
2,-d5MIcv7Gtetq5g1xLaolc5qUozE9oX3ogQQJCLK2RCUv_...,2109058.0,197.0,50.0,0.0,14.0,18.0,137149.0,8372.0,12921.0,...,0.0,0.0,0.0,0.0,0.0,213.0,127.0,147.0,0.0,0.0
3,13DRof6AgQVW9VmqftA-XWar4Fxf-tNxD0mmubwlEZwQtS...,1557593.0,541.0,0.0,8.0,16.0,227.0,178260.0,12160.0,15490.0,...,0.0,0.0,0.0,0.0,0.0,517.0,168.0,0.0,0.0,0.0
4,1ERlFiDMWvXUJsYSTEdjmft0_R6aw2eiuahq5TRSLQidfN...,1429418.0,417.0,50.0,0.0,10.0,43.0,99430.0,6492.0,6738.0,...,0.0,0.0,0.0,0.0,0.0,215.0,173.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,uqyG5YZ07CbimBvEtjCXwbzhONsABM9fGTlQC5x1ZClvHH...,2076960.0,483.0,50.0,0.0,13.0,26.0,442968.0,9083.0,11379.0,...,0.0,0.0,0.0,0.0,0.0,217.0,207.0,0.0,0.0,0.0
72,v4iBqXC1o7TN__NBmM65HlUOPRvxagalg8_du9vH8YtYN0...,1557593.0,412.0,50.0,0.0,11.0,31.0,234731.0,7537.0,8415.0,...,0.0,0.0,0.0,0.0,0.0,202.0,285.0,0.0,0.0,0.0
73,wWFgTMjxXdNPJgVCWg8S3bIexkChFWKijPLdYUkZmqR80h...,1557593.0,1209.0,0.0,5.0,15.0,173.0,31183.0,11419.0,13679.0,...,0.0,0.0,0.0,0.0,0.0,481.0,133.0,134.0,0.0,0.0
74,yrrz30zFwu5ODiWSm1ORwtjaBy4gf5XPLF61uDCkRpqF6X...,2038851.0,1351.0,0.0,16.0,18.0,279.0,452288.0,15300.0,19953.0,...,0.0,0.0,0.0,0.0,0.0,234.0,149.0,466.0,0.0,0.0


## 3. 변수 선택 및 스케일링

In [20]:
features = ['puuid',
    "lane_cs_per_min", "jungle_cs_per_min",
    "KDA", "gold_per_min", "xp_per_min",
    "damage_per_min", "damage_taken_per_min", "ka_per_min", "damage_efficiency",
    "survivability", "healthregen_per_min",
    "offensive_score", "defensive_score", "magic_score", "total_vamp",
    "wards_placed_per_min", "wards_killed_per_min"
]

In [21]:
filtered_df = grouped_df[features].fillna(0)
filtered_df

,puuid,lane_cs_per_min,jungle_cs_per_min,KDA,gold_per_min,xp_per_min,damage_per_min,damage_taken_per_min,ka_per_min,damage_efficiency,survivability,healthregen_per_min,offensive_score,defensive_score,magic_score,total_vamp,wards_placed_per_min,wards_killed_per_min
0,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,1.546045,3.788314,1.804935,429.875512,473.095802,798.382411,1148.428457,0.293057,0.682956,0.243539,91.022159,398.9,190.3,114.7,0.0,0.0,0.0
1,-J6KMZ7IDQZNRHXtuMzSQQRkDwinSGI62fo5qD6C700Hzr...,1.252599,0.000000,0.500000,239.434326,249.267230,188.390911,454.630859,0.062630,0.414382,0.500000,50.041336,182.0,99.0,29.0,0.0,0.0,0.0
2,-d5MIcv7Gtetq5g1xLaolc5qUozE9oX3ogQQJCLK2RCUv_...,0.512077,0.000000,0.000000,238.172682,367.585908,203.351449,706.410160,0.000000,0.287866,0.100000,30.923758,213.0,127.0,147.0,0.0,0.0,0.0
3,13DRof6AgQVW9VmqftA-XWar4Fxf-tNxD0mmubwlEZwQtS...,8.744261,0.308168,1.666667,468.415048,596.689893,604.124441,887.176560,0.192605,0.680952,0.333333,65.485656,517.0,168.0,0.0,0.0,0.0,0.0
4,1ERlFiDMWvXUJsYSTEdjmft0_R6aw2eiuahq5TRSLQidfN...,1.804930,0.000000,1.000000,272.502515,282.828396,244.547081,459.291824,0.083950,0.532444,0.500000,40.715872,215.0,173.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,uqyG5YZ07CbimBvEtjCXwbzhONsABM9fGTlQC5x1ZClvHH...,0.751098,0.000000,0.500000,262.393113,328.720823,213.369540,882.366536,0.057777,0.241815,0.250000,217.876127,217.0,207.0,0.0,0.0,0.0,0.0
72,v4iBqXC1o7TN__NBmM65HlUOPRvxagalg8_du9vH8YtYN0...,1.194150,0.000000,0.333333,290.332584,324.153999,254.777724,562.136579,0.077042,0.453231,0.166667,51.348459,202.0,285.0,0.0,0.0,0.0,0.0
73,wWFgTMjxXdNPJgVCWg8S3bIexkChFWKijPLdYUkZmqR80h...,6.664129,0.192605,1.750000,439.871006,526.928408,892.800622,581.666713,0.269647,1.534901,0.250000,22.958501,481.0,133.0,134.0,0.0,0.0,0.0
74,yrrz30zFwu5ODiWSm1ORwtjaBy4gf5XPLF61uDCkRpqF6X...,8.210507,0.470853,1.400000,450.253599,587.183664,621.732535,920.812752,0.205998,0.675200,0.200000,50.763886,234.0,149.0,466.0,0.0,0.0,0.0


In [22]:
filtered_df.isnull().sum()

puuid                   0
lane_cs_per_min         0
jungle_cs_per_min       0
KDA                     0
gold_per_min            0
xp_per_min              0
damage_per_min          0
damage_taken_per_min    0
ka_per_min              0
damage_efficiency       0
survivability           0
healthregen_per_min     0
offensive_score         0
defensive_score         0
magic_score             0
total_vamp              0
wards_placed_per_min    0
wards_killed_per_min    0
dtype: int64

In [23]:
# 스케일링
scaler = StandardScaler()

# 분석 대상에서 제외할 컬럼 지정
id_columns = ['puuid']
features = filtered_df.drop(columns=id_columns)

# StandardScaler 적용
scaler = StandardScaler()
scaled_array = scaler.fit_transform(features)

# 정규화된 데이터를 다시 DataFrame으로
scaled_features = pd.DataFrame(scaled_array, columns=features.columns, index=features.index)

# 식별자 컬럼 붙이기
scaled_df = pd.concat([filtered_df[id_columns], scaled_features], axis=1)

scaled_df

,puuid,lane_cs_per_min,jungle_cs_per_min,KDA,gold_per_min,xp_per_min,damage_per_min,damage_taken_per_min,ka_per_min,damage_efficiency,survivability,healthregen_per_min,offensive_score,defensive_score,magic_score,total_vamp,wards_placed_per_min,wards_killed_per_min
0,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,-1.036673,1.466676,0.414888,0.449370,0.206065,0.286351,0.627821,0.696177,-0.264621,-0.167017,1.715338,0.145067,0.076626,0.047672,0.0,0.0,0.0
1,-J6KMZ7IDQZNRHXtuMzSQQRkDwinSGI62fo5qD6C700Hzr...,-1.133717,-0.518764,-0.552861,-1.543108,-2.189635,-1.324364,-1.346892,-0.807420,-0.809331,0.868239,0.239653,-1.230387,-1.206589,-0.429174,0.0,0.0,0.0
2,-d5MIcv7Gtetq5g1xLaolc5qUozE9oX3ogQQJCLK2RCUv_...,-1.378611,-0.518764,-0.923665,-1.556308,-0.923237,-1.284860,-0.630268,-1.216098,-1.065926,-0.746441,-0.448755,-1.033803,-0.813051,0.227393,0.0,0.0,0.0
3,13DRof6AgQVW9VmqftA-XWar4Fxf-tNxD0mmubwlEZwQtS...,1.343813,-0.357255,0.312347,0.852587,1.528927,-0.226598,-0.115764,0.040701,-0.268686,0.195456,0.795790,0.893989,-0.236799,-0.590533,0.0,0.0,0.0
4,1ERlFiDMWvXUJsYSTEdjmft0_R6aw2eiuahq5TRSLQidfN...,-0.951058,-0.518764,-0.182058,-1.197134,-1.830420,-1.176081,-1.333625,-0.668300,-0.569883,0.868239,-0.096149,-1.021121,-0.166524,-0.590533,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,uqyG5YZ07CbimBvEtjCXwbzhONsABM9fGTlQC5x1ZClvHH...,-1.299566,-0.518764,-0.552861,-1.302903,-1.339221,-1.258407,-0.129455,-0.839089,-1.159325,-0.140936,6.283243,-1.008438,0.311344,-0.590533,0.0,0.0,0.0
72,v4iBqXC1o7TN__NBmM65HlUOPRvxagalg8_du9vH8YtYN0...,-1.153046,-0.518764,-0.676462,-1.010588,-1.388101,-1.149066,-1.040905,-0.713378,-0.730539,-0.477327,0.286721,-1.103559,1.407629,-0.590533,0.0,0.0,0.0
73,wWFgTMjxXdNPJgVCWg8S3bIexkChFWKijPLdYUkZmqR80h...,0.655903,-0.417821,0.374147,0.553947,0.782251,0.535667,-0.985318,0.543420,1.463259,-0.140936,-0.735577,0.665698,-0.728722,0.155059,0.0,0.0,0.0
74,yrrz30zFwu5ODiWSm1ORwtjaBy4gf5XPLF61uDCkRpqF6X...,1.167298,-0.271992,0.114585,0.662574,1.427180,-0.180103,-0.020028,0.128097,-0.280352,-0.342771,0.265671,-0.900634,-0.503843,2.002348,0.0,0.0,0.0


In [24]:

target_puuid = '-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFXsxfHwXECJ_bWG0f57TpkVxUgYiRdb85Q' 
X_target = scaled_df[scaled_df["puuid"] == target_puuid].fillna(0)
X_target.to_csv('data/result/normal.csv')
X_target

,puuid,lane_cs_per_min,jungle_cs_per_min,KDA,gold_per_min,xp_per_min,damage_per_min,damage_taken_per_min,ka_per_min,damage_efficiency,survivability,healthregen_per_min,offensive_score,defensive_score,magic_score,total_vamp,wards_placed_per_min,wards_killed_per_min
0,-356wHRenGKjOp5pBnrCzWJv7qFR5ypK9iMpB1S0J8doFX...,-1.036673,1.466676,0.414888,0.44937,0.206065,0.286351,0.627821,0.696177,-0.264621,-0.167017,1.715338,0.145067,0.076626,0.047672,0.0,0.0,0.0
